In [1]:
#Standard Imports
import os
import sys
import pandas as pd
# from collections import defaultdict
#import timeit
from docplex.cp.model import CpoModel

In [2]:
#Custom Imports
sys.path.append('../')
from src.chord import Chord
from src.musical_work_input import MusicalWorkInput
from src.cp_model import *
from src.music_functions import *
from src.evaluate import *
from src.midi_processing import *

In [3]:
# Importing Musical Corpus
musical_work_df = pd.read_csv("../data/sample_input.csv")
musical_corpus = []
for i, title, meter, key, tonality, first_on_beat, melody in musical_work_df.itertuples():
    musical_corpus.append(MusicalWorkInput(title, meter, key, tonality, first_on_beat, [int(x) for x in melody.split(',')]))

In [4]:
# Importing Chord Vocabulary
chord_df_major = pd.read_csv("../data/chord_vocabulary_major.csv", index_col = 0)
chord_df_minor = pd.read_csv("../data/chord_vocabulary_minor.csv", index_col = 0)
chord_vocab_major, chord_vocab_minor = [], []
for index, name, note_intervals in chord_df_major.itertuples():
    chord_vocab_major.append(Chord(index, name, [int(x) for x in note_intervals.split(',')]))
for index, name, note_intervals in chord_df_minor.itertuples():
    chord_vocab_minor.append(Chord(index, name, [int(x) for x in note_intervals.split(',')]))

In [5]:
# Defining penalties for chord progression
penalties_chord_progression_major = pd.read_csv("../data/chord_progression_major.csv", header = 1, index_col = 0)
penalties_chord_progression_minor = pd.read_csv("../data/chord_progression_minor.csv", header = 1, index_col = 0)
penalties_chord_progression_major = dict(penalties_chord_progression_major.stack())
penalties_chord_progression_minor = dict(penalties_chord_progression_minor.stack())

In [6]:
# Importing Weights
weight_df = pd.read_csv("../data/soft_constraint_weights_cp2.csv")

# Defining dictionary of weights for each soft constraint option:
soft_constraint_w_weights = {}
for _, name, w in weight_df.itertuples(): #name population is same as soft_constraint_options
    soft_constraint_w_weights[name] = float(w)
assert sum(v for v in soft_constraint_w_weights.values() if v > 0) == 100

In [7]:
# Defining dictionary of hard and soft constraint options:
hard_constraint_options = ['musical input', 'voice range', 'chord membership', 'first last chords',
                           'chord repetition', 'chord bass repetition', 'adjacent bar chords', 'voice crossing', 'parallel movement',
                          'chord spacing', 'incomplete chord', 'chord distribution']
soft_constraint_options = ['chord progression', 'chord repetition', 'chord bass repetition', 'leap resolution',
                           'melodic movement', 'note repetition', 'parallel movement', 'voice overlap', 'adjacent bar chords',
                           'chord spacing', 'distinct notes', 'incomplete chord', 'voice crossing', 'voice range',
                           'second inversion', 'first inversion', 'chord distribution']

In [8]:
# Defining which hard constraints to use
hard_constraints = {x: True if x in ['musical input', 'voice range', 'chord membership', 'first last chords',
                                     'chord bass repetition', 'adjacent bar chords',
                                     'voice crossing', 'parallel movement',
                                     'chord spacing', 'incomplete chord'] else False for x in hard_constraint_options}

In [9]:
musical_corpus

In [10]:
# Model
musical_work = musical_corpus[-1]

print('*'*20)
print('Work: {}'.format(musical_work.title))
print('*'*20)
    
if musical_work.tonality == 'major':
    penalties_chord_progression = penalties_chord_progression_major
    chord_vocab = chord_vocab_major
else:
    penalties_chord_progression = penalties_chord_progression_minor
    chord_vocab = chord_vocab_minor        
    
#Defining Model
cp_model = CPModel('Completed_v2', musical_work, chord_vocab, penalties_chord_progression,
                    hard_constraints,
                    soft_constraint_w_weights)
    
#Solving Model
# solution, progress_data = cp_model.solve(log_output = True, TimeLimit = 30, LogVerbosity = 'Verbose')
#solution = cp_model.solve_old(log_output = True, TimeLimit = 30, LogVerbosity = 'Verbose')
# result = cp_model.get_solution()


********************
Work: Ach Gott und Herr, wie gross und schwer (16)
********************


In [11]:
solution, progress_data = cp_model.solve(log_output = True, TimeLimit = 600, LogVerbosity = 'Verbose')

In [12]:
print(progress_data)

[(1.2329976558685303, 265), (1.5429930686950684, 263), (1.904994010925293, 260), (2.215993881225586, 248), (2.3789923191070557, 220), (2.9669933319091797, 216), (3.587993621826172, 22), (4.102073431015015, 20), (4.32307767868042, 18)]


In [ ]:
'''list_x = cp_model.sol_var['Notes']
list_c = []
for chord_name in cp_model.sol_var['Chords']:
    exists = False
    for chord in chord_vocab:
        if chord.name == chord_name:
            exists = True
            list_c.append(chord.index)
            break
    if not exists:
        list_c.append(-1)'''

In [ ]:
'''total_cp_cost = evaluate_cost(list_x, list_c, cp_model.K, cp_model.musical_input.tonality,
                     cp_model.musical_input.meter, cp_model.musical_input.first_on_beat, 'D',
                     chord_vocab,
                     penalties_chord_progression,
                     hard_constraints, 1000,
                     soft_constraint_w_weights)
print(sum(total_cp_cost.values()))'''